In [ ]:
!git clone https://github.com/demyanchuk-nestor/Low_Data_Drug_Discovery
%cd 'Low_Data_Drug_Discovery'

In [7]:
from IPython.display import clear_output

# https://www.dgl.ai/pages/start.html

# !pip install dgl
# !pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html # FOR CUDA VERSION
!pip install  dgl -f https://data.dgl.ai/wheels/cu121/repo.html
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install chembl_structure_pipeline
!pip install dgllife
!pip install rdkit-pypi
!pip install --pre deepchem
!pip install ipython-autotime
!pip install gputil
!pip install psutil
!pip install humanize

!conda install -c chembl chembl_structure_pipeline

%load_ext autotime

clear = clear_output()

time: 44.9 s (started: 2024-03-29 08:19:18 +00:00)


In [11]:
import os
from os import path
import pandas as pd
import numpy as np
import statistics
import warnings
import random
import time
import itertools

import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
import tqdm
from tqdm import trange
import sklearn
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

import dgl
from dgl.dataloading import GraphDataLoader
from dgl.nn import GraphConv, SumPooling, MaxPooling
import dgl.function as fn
import dgllife
from dgllife import utils

import deepchem as dc
import rdkit
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize

from torch.utils.tensorboard import SummaryWriter

import chembl_structure_pipeline
from chembl_structure_pipeline import standardizer

# embedding
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from tqdm import tqdm_notebook
from tqdm import tnrange

# from torch.autograd import Variable

# Profiler
from torch.profiler import profile, record_function, ProfilerActivity

import psutil
import humanize
import GPUtil as GPU
import subprocess

from datetime import datetime, timedelta

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import PrecisionRecallDisplay

warnings.filterwarnings("ignore", message="DGLGraph.__len__")

DGLBACKEND = 'pytorch'

clear

def get_cmd_output(command):
  return subprocess.check_output(command,
                        stderr=subprocess.STDOUT,
                        shell=True).decode('UTF-8')

time: 2.59 ms (started: 2024-03-29 08:21:38 +00:00)


In [12]:
def create_dataset(df, name, bonds):
    print(f"Creating Dataset and Saving to {drive_path}/data/{name}.pkl")
    data = df.sample(frac=1)
    data = data.reset_index(drop=True)
    data['mol'] = data['smiles'].apply(lambda x: create_dgl_features(x, bonds))
    data.to_pickle(f"{drive_path}/data/{name}.pkl")
    return data

def create_ecfp_dataset(df, name, bonds):
    print(f"Creating Dataset and Saving to {drive_path}/data/{name}.pkl")
    data = df.sample(frac=1)
    data = data.reset_index(drop=True)

    featurizer = dc.feat.CircularFingerprint(size=2048, radius=2)

    data['mol'] = data['smiles'].apply(lambda x: np.float32(featurizer(x)[0]))
    data.to_pickle(f"{drive_path}/data/{name}.pkl")
    return data

def featurize_atoms(mol):
  feats = []

  atom_features = utils.ConcatFeaturizer([
    utils.atom_type_one_hot,
    utils.atomic_number_one_hot,
    utils.atom_degree_one_hot,
    utils.atom_explicit_valence_one_hot,
    utils.atom_formal_charge_one_hot,
    utils.atom_num_radical_electrons_one_hot,
    utils.atom_hybridization_one_hot,
    utils.atom_is_aromatic_one_hot
  ])

  for atom in mol.GetAtoms():
    feats.append(atom_features(atom))
  return {'feats': torch.tensor(feats).float()}

def featurize_bonds(mol):
  feats = []

  bond_features = utils.ConcatFeaturizer([
    utils.bond_type_one_hot,
    utils.bond_is_conjugated_one_hot,
    utils.bond_is_in_ring_one_hot,
    utils.bond_stereo_one_hot,
    utils.bond_direction_one_hot,
  ])

  for bond in mol.GetBonds():
    feats.append(bond_features(bond))
    feats.append(bond_features(bond))
  return {'edge_feats': torch.tensor(feats).float()}

def create_dgl_features(smiles, bonds):
    mol = Chem.MolFromSmiles(smiles)
    mol = standardizer.standardize_mol(mol)

    if bonds:
        dgl_graph = utils.mol_to_bigraph(mol=mol,
                                    node_featurizer=featurize_atoms,
                                    edge_featurizer=featurize_bonds,
                                    canonical_atom_order=True)
    else:
      dgl_graph = utils.mol_to_bigraph(mol=mol,
                                    node_featurizer=featurize_atoms,
                                    canonical_atom_order=True)

    dgl_graph = dgl.add_self_loop(dgl_graph)

    return dgl_graph

def load_dataset(dataset, bonds=False, feat='graph', create_new=False):
  """
    dataset values: muv, tox21
  """
  dataset_test_tasks = {
    'tox21': ['SR-HSE', 'SR-MMP', 'SR-p53'],
    'muv': ['MUV-832', 'MUV-846', 'MUV-852', 'MUV-858', 'MUV-859'],
    'dude-gpcr': ['adrb2', 'cxcr4']
  }

  dataset_original = dataset

  if bonds:
    dataset = dataset + "_with_bonds"

  if path.exists(f"{drive_path}/data/{dataset}_dgl.pkl") and not create_new:
    # Load Dataset
    print("Reading Pickle")
    if feat == 'graph':
      data = pd.read_pickle(f"{drive_path}/data/{dataset}_dgl.pkl")
    else:
      data = pd.read_pickle(f"{drive_path}/data/{dataset}_ecfp.pkl")
  else:
    # Create Dataset
    df = pd.read_csv(f"{drive_path}/data/raw/{dataset_original}.csv")
    if feat == 'graph':
      data = create_dataset(df, f"{dataset}_dgl", bonds)
    else:
      data = create_ecfp_dataset(df, f"{dataset}_ecfp")

  test_tasks = dataset_test_tasks.get(dataset_original)
  drop_cols = test_tasks.copy()
  drop_cols.extend(['mol_id', 'smiles', 'mol'])
  train_tasks = [x for x in list(data.columns) if x not in drop_cols]

  return data, train_tasks, test_tasks

time: 4.54 ms (started: 2024-03-29 08:21:42 +00:00)


In [18]:
drive_path = "./data"

# PARAMETERS
dataset = 'dude-gpcr'
with_bonds = False

df = pd.read_csv(f"{drive_path}/raw/{dataset}.csv")


# data = create_dataset(df, f"{dataset}_dgl", bonds)
data = create_ecfp_dataset(df, f"{dataset}_ecfp", with_bonds)

Creating Dataset and Saving to ./data/data/dude-gpcr_ecfp.pkl
time: 2min 15s (started: 2024-03-29 08:32:43 +00:00)
